In [ ]:
import sys
import shutil
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import flopy

### Set executable names

In [ ]:
mf6exe = 'mf6'
swtexe = 'swtv4'

### Levels for contours

In [ ]:
levels = [0.01, .1, .5, .9, .99]

### Model dimensions

In [ ]:
nper = 1
perlen = 0.5
nstp = 500
tsmult = 1.
steady = True

In [ ]:
nlay, nrow, ncol = 40, 1, 80
lengthx, lengthz = 2., 1.
delr = lengthx / ncol
delz = lengthz / nlay
delc = 1.

In [ ]:
top = 1.
botm = list(top - np.arange(delz, nlay * delz + delz, delz))

In [ ]:
qinflow = 5.7024
constant_concentration = True

### Build a SEAWAT model

In [ ]:
# SEAWAT model
ws = "temp/henry_seawat"
name = "henry"

In [ ]:
swt = flopy.seawat.Seawat(name, exe_name=swtexe, model_ws=ws, )
dis = flopy.modflow.ModflowDis(swt, nlay, nrow, ncol, nper=1, delr=delr,
                               delc=delc, laycbd=0, top=top, botm=botm, 
                               perlen=perlen, nstp=1, steady=steady)
ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)
ibound[:, :, -1] = -1
bas = flopy.modflow.ModflowBas(swt, ibound, 1)
lpf = flopy.modflow.ModflowLpf(swt, hk=864., vka=864.)
pcg = flopy.modflow.ModflowPcg(swt, hclose=1.e-8)
oc = flopy.modflow.ModflowOc(swt, 
                             stress_period_data={(0, 0): ['save head', 'save budget']},
                             compact=True)
itype = flopy.mt3d.Mt3dSsm.itype_dict()
wel_data = {}
ssm_data = {}
wel_sp1 = []
ssm_sp1 = []
for k in range(nlay):
    wel_sp1.append([k, 0, 0, qinflow / nlay])
    ssm_sp1.append([k, 0, 0, 0., itype['WEL']])
    ssm_sp1.append([k, 0, ncol - 1, 35., itype['BAS6']])
wel_data[0] = wel_sp1
ssm_data[0] = ssm_sp1
wel = flopy.modflow.ModflowWel(swt, stress_period_data=wel_data)
if constant_concentration:
    icbund = ibound
else:
    icbund = 1
btn = flopy.mt3d.Mt3dBtn(swt, icbund=icbund, nprs=-5, prsity=0.35, sconc=35., ifmtcn=0,
                         chkmas=False, nprobs=10, nprmas=10, dt0=perlen/500.)
adv = flopy.mt3d.Mt3dAdv(swt, mixelm=0)
dsp = flopy.mt3d.Mt3dDsp(swt, al=0., trpt=1., trpv=1., dmcoef=0.57024)
gcg = flopy.mt3d.Mt3dGcg(swt, iter1=500, mxiter=1, isolve=1, cclose=1e-7)
ssm = flopy.mt3d.Mt3dSsm(swt, stress_period_data=ssm_data)

vdf = flopy.seawat.SeawatVdf(swt, iwtable=0, densemin=0, densemax=0,
                             denseref=1000., denseslp=0.7, firstdt=perlen/500.)

swt.write_input()
v = swt.run_model(silent=True, report=True)
for idx in range(-3, 0):
    print(v[1][idx])

### Post process the results

In [ ]:
fpth = os.path.join(ws, 'MT3D001.UCN')
concobj = flopy.utils.UcnFile(fpth)
conc_swt = concobj.get_data(totim=concobj.get_times()[-1])
concobj.file.close()

### Plot the SEAWAT results

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
xs = flopy.plot.PlotCrossSection(swt, line={'row': 0})
xs.plot_grid(linewidths=0.5, alpha=0.5)
xs.plot_array(conc_swt, cmap='jet', vmin=0, vmax=35)
cs = xs.contour_array(conc_swt / 35., levels=levels, colors='white')
ax.clabel(cs)
ax.set_xlabel('DISTANCE, IN METERS')
ax.set_ylabel('ELEVATION, IN METERS')
ax.set_title('SEAWAT');

### Build the MODFLOW 6 model

In [ ]:
ws = "temp/henry_mf6"
name = "henry"

In [ ]:
nouter, ninner = 100, 300
hclose, rclose, relax = 1e-7, 1e-5, 0.97

#### set the names of the gwf and gwt models

In [ ]:
gwfname = 'gwf_' + name
gwtname = 'gwt_' + name

#### Build the groundwater flow model

In [ ]:
# build MODFLOW 6 files
sim = flopy.mf6.MFSimulation(sim_name=name, version='mf6',
                             exe_name=mf6exe,
                             sim_ws=ws)
# create tdis package
tdis = flopy.mf6.ModflowTdis(sim, time_units='DAYS',
                             nper=nper, perioddata=[perlen, nstp, tsmult])

# create gwf model
gwf = flopy.mf6.MFModel(sim, model_type='gwf6', modelname=gwfname)

imsgwf = flopy.mf6.ModflowIms(sim, print_option='ALL',
                              outer_dvclose=hclose,
                              outer_maximum=nouter,
                              under_relaxation='NONE',
                              inner_maximum=ninner,
                              inner_dvclose=hclose, rcloserecord=rclose,
                              linear_acceleration='BICGSTAB',
                              scaling_method='NONE',
                              reordering_method='NONE',
                              relaxation_factor=relax,
                              filename='{}.ims'.format(gwfname))
sim.register_ims_package(imsgwf, [gwfname])

dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,
                              delr=delr, delc=delc,
                              top=top, botm=botm)

# initial conditions
ic = flopy.mf6.ModflowGwfic(gwf, strt=1.)

# node property flow
npf = flopy.mf6.ModflowGwfnpf(gwf, 
                              save_flows=True,
                              save_specific_discharge=True,
                              icelltype=0,
                              k=864.)

buy = flopy.mf6.ModflowGwfbuy(
    gwf, packagedata=[(0, 0.7, 0.0, gwtname, "concentration")],
)

def chd_value(k):
    f = 0.
    depth = k * delz + 0.5 * delz
    hf = top + f * depth
    return hf

# chd files
chdlist1 = []
for k in range(nlay):
    chdlist1.append([(k, 0, ncol - 1), chd_value(k), 35.])
chd1 = flopy.mf6.ModflowGwfchd(gwf,
                               stress_period_data=chdlist1,
                               print_input=True,
                               print_flows=True,
                               save_flows=False,
                               pname='CHD-1',
                               auxiliary='CONCENTRATION')

wellist1 = []
qwell = qinflow / nlay
for k in range(nlay):
    wellist1.append([(k, 0, 0), qwell, 0.])
wel1 = flopy.mf6.ModflowGwfwel(gwf,
                               stress_period_data=wellist1,
                               print_input=True,
                               print_flows=True,
                               save_flows=False,
                               pname='WEL-1',
                               auxiliary='CONCENTRATION')

# output control
oc = flopy.mf6.ModflowGwfoc(gwf,
                            budget_filerecord='{}.cbc'.format(gwfname),
                            head_filerecord='{}.hds'.format(gwfname),
                            headprintrecord=[
                                ('COLUMNS', 10, 'WIDTH', 15,
                                 'DIGITS', 6, 'GENERAL')],
                            saverecord=[('HEAD', 'LAST'),
                                        ('BUDGET', 'LAST')],
                            printrecord=[('HEAD', 'LAST'),
                                         ('BUDGET', 'LAST')])

#### Build the groundwater transport model

In [ ]:
# create gwt model
gwt = flopy.mf6.MFModel(sim, model_type='gwt6', modelname=gwtname,
                        model_nam_file='{}.nam'.format(gwtname))

imsgwt = flopy.mf6.ModflowIms(sim, print_option='ALL',
                              outer_dvclose=hclose,
                              outer_maximum=nouter,
                              under_relaxation='NONE',
                              inner_maximum=ninner,
                              inner_dvclose=hclose, rcloserecord=rclose,
                              linear_acceleration='BICGSTAB',
                              scaling_method='NONE',
                              reordering_method='NONE',
                              relaxation_factor=relax,
                              filename='{}.ims'.format(gwtname))
sim.register_ims_package(imsgwt, [gwt.name])

dis = flopy.mf6.ModflowGwtdis(gwt, nlay=nlay, nrow=nrow, ncol=ncol,
                              delr=delr, delc=delc,
                              top=top, botm=botm)

# initial conditions
ic = flopy.mf6.ModflowGwtic(gwt, strt=35.)

# advection
adv = flopy.mf6.ModflowGwtadv(gwt, scheme='UPSTREAM')

# dispersion
diffc = 0.57024
dsp = flopy.mf6.ModflowGwtdsp(gwt, diffc=diffc,
                              xt3d_off=True)

# mass storage and transfer
porosity = 0.35
mst = flopy.mf6.ModflowGwtmst(gwt, porosity=porosity)

# sources
sourcerecarray = [('CHD-1', 'AUX', 'CONCENTRATION'),
                  ('WEL-1', 'AUX', 'CONCENTRATION')]
ssm = flopy.mf6.ModflowGwtssm(gwt, sources=sourcerecarray)

# constant concentration
if constant_concentration:
    cnclist = []
    for k in range(nlay):
        cnclist.append([(k, 0, ncol - 1), 35.])
    cnc = flopy.mf6.ModflowGwtcnc(gwt, stress_period_data=cnclist)

# output control
oc = flopy.mf6.ModflowGwtoc(gwt,
                            budget_filerecord='{}.cbc'.format(gwtname),
                            concentration_filerecord='{}.ucn'.format(
                                gwtname),
                            concentrationprintrecord=[
                                ('COLUMNS', 10, 'WIDTH', 15,
                                 'DIGITS', 6, 'GENERAL')],
                            saverecord=[('CONCENTRATION', 'ALL')],
                            printrecord=[('CONCENTRATION', 'LAST'),
                                         ('BUDGET', 'LAST')])

# GWF GWT exchange
gwfgwt = flopy.mf6.ModflowGwfgwt(sim, exgtype='GWF6-GWT6',
                                 exgmnamea=gwfname, exgmnameb=gwtname)

### Write the model files and run the simulation

In [ ]:
sim.write_simulation(silent=True)
v = sim.run_simulation(silent=True, report=True)
for idx in range(-3, 0):
    print(v[1][idx])

### Post process the MODFLOW 6 results

In [ ]:
concobj = gwt.output.concentration()
times = concobj.get_times()
conc = concobj.get_data(totim=times[-1])

### Plot the MODFLOW 6 results

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
xs = flopy.plot.PlotCrossSection(gwf, line={'row': 0})
xs.plot_grid(linewidths=0.5, alpha=0.5)
xs.plot_array(conc, cmap='jet', vmin=0, vmax=35)
cs = xs.contour_array(conc / 35., levels=levels, colors='white')
ax.clabel(cs)
ax.set_xlabel('DISTANCE, IN METERS')
ax.set_ylabel('ELEVATION, IN METERS')
title = 'MODFLOW 6 '
ax.set_title("HYDRAULIC HEAD FORMULATION");

### Compare the contours

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
xs = flopy.plot.PlotCrossSection(gwf, line={'row': 0})
cs = xs.contour_array(conc / 35., levels=levels, colors='black', linewidths=4)
ax.clabel(cs)
cs = xs.contour_array(conc_swt / 35., levels=levels, colors='red', linewidths=1.5, linestyles='--')
#ax.clabel(cs)

ax.set_xlabel('DISTANCE, IN METERS')
ax.set_ylabel('ELEVATION, IN METERS');
ax.set_title('MODFLOW 6 - SEAWAT COMPARISON');